In [12]:
import xgboost as xgb
import pandas as pd
import numpy as np


In [13]:
df = pd.read_csv('filtered_nn_data.csv')

print(df.head())

         date  year      uv  buyer    gmv  orders     cr  day_of_week  \
0  2021-01-01  2021   96281   3595  66052    4331  0.037            4   
1  2021-01-02  2021   93620   3852  68483    4599  0.041            5   
2  2021-01-03  2021   98196   4402  75556    5254  0.045            6   
3  2021-01-04  2021  108903   4374  75432    5292  0.040            0   
4  2021-01-05  2021  123715   4486  74678    5518  0.036            1   

   weekday day_type  day_binary campaign_info campaign_type  
0        1  Weekday         100  non campaign  non-campaign  
1        0  Weekend         101  non campaign  non-campaign  
2        0  Weekend         110  non campaign  non-campaign  
3        1  Weekday           0  non campaign  non-campaign  
4        1  Weekday           1  non campaign  non-campaign  


In [14]:
df = df[['date',  'orders']].copy()
# print('\n', 'uv', 'buyer', 'gmv',)
print(df)

           date  orders
0    2021-01-01    4331
1    2021-01-02    4599
2    2021-01-03    5254
3    2021-01-04    5292
4    2021-01-05    5518
..          ...     ...
743  2023-08-18    9768
744  2023-08-19    9740
745  2023-08-20   10201
746  2023-08-21    9820
747  2023-08-22    9974

[748 rows x 2 columns]


In [15]:
# Create lag features (e.g., lag 1 day)
df['lag_1_day'] = df['orders'].shift(1)

In [23]:
# df['date'] = pd.to_datetime(df['date'])
# Initialize and train the XGBoost model using all available data
# Verify data type of 'orders'
print(df['orders'].dtype)  # Check if it's int or float

# Initialize and train the XGBoost model using all available data
X = df[['lag_1_day']].dropna()
y = df['orders'].dropna()

# Align 'X' and 'y' by dropping the last row from 'y'
y = y[:-1]  # Remove the last row

# Debugging: Print shapes of X and y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(X, y)  # Fit the model before making predictions

# Create a list to store forecasted data
forecast_data = []

# Set the initial date to the last date in your dataset
initial_date = df['date'].max()

# Forecast for 365 days into the future
for day in range(1, 366):
    # Create a DataFrame with the lagged order value for prediction
    prediction_data = pd.DataFrame({'lag_1_day': [y.iloc[-1]]})
    
    # Predict the order for the next day
    next_day_order = model.predict(prediction_data)
    
    # Convert DateOffset to Timedelta and add it to the date
    next_date = initial_date + pd.Timedelta(days=day)
    
    # Append data to the forecast list
    forecast_data.append({'date': next_date, 'orders': next_day_order[0]})

    # Update the lagged order value for the next iteration
    y = y.append(pd.Series(next_day_order[0]), ignore_index=True)

# Convert the list to a DataFrame
forecast_df = pd.DataFrame(forecast_data)

# Display the forecasted orders for 365 days
print(forecast_df)

int64
Shape of X: (747, 1)
Shape of y: (747,)


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinsta

TypeError: can only concatenate str (not "Timedelta") to str